<a href="https://colab.research.google.com/github/mech0s/nodehenge/blob/main/experiment/experiment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
revision = 5
print ("Revision", revision)

Revision 5


##  Install steps
Remeber to re-run for each fresh google colab session

In [1]:
%pip install rdfpandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.


##   Imports

In [3]:
import numpy as np
import pandas as pd


### Optional    google  colab   enhancements

In [2]:
#optional
from google.colab import data_table
data_table.enable_dataframe_formatter()

##  Code

In [4]:
sourceURI = "https://dodcio.defense.gov/Portals/0/Documents/Library/DevSecOpsActivitesToolsGuidebookTables.xlsx"
xls = pd.ExcelFile(sourceURI)

In [5]:
xls.sheet_names

['Cover',
 'Overview',
 'Continuous Activities',
 'Plan',
 'Develop',
 'Build',
 'Test',
 'Release',
 'Deliver',
 'Deploy',
 'Operate',
 'Monitor',
 'Feedback',
 'Tools']

In [6]:
phaseNames = [ 'Plan',
 'Develop',
 'Build',
 'Test',
 'Release',
 'Deliver',
 'Deploy',
 'Operate',
 'Monitor',
 'Feedback']

One named worksheet per phase: Pull these into a list of DataFrames, adjusting column names to create valid identifiers. Turn NaN entries into blank strings.

In [7]:
phaseDataList = []
for phName in phaseNames:
  df = pd.read_excel(xls,phName)
  df = df.rename(columns={"Activities":"Activity",
                     "Security / Testing / CM": "SecurityTestingCM",
                     "Tool Dependencies": "ToolDependency",
                     "Tool Dependency": "ToolDependency"
                     })
  phaseDataList.append(df.replace(np.nan, ""))

List of dataframes - concatenated into one dataframe, with the addition of a Phase column for identification.

In [8]:
phaseActivityDataFrame = pd.concat(phaseDataList,keys=phaseNames, names=["Phase","IDinPhase"])

In [ ]:
phaseActivityDataFrame

In [19]:
tmpdf =  phaseActivityDataFrame.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)




In [ ]:
variableColumns = [
                   #'Activity' ,
                   "SSDF",
                    'Baseline',
                    'Description',
                    'Inputs',
                    'Outputs',
                   # 'SecurityTestingCM',
                   # 'ToolDependency'
                   ]

tmpdf.columns.difference(variableColumns)
tmpdf[tmpdf.duplicated(keep=False, subset=tmpdf.columns.difference(variableColumns) )]

In [12]:
variableColumns = [ ]#,'Description']
phaseActivityDataFrame[phaseActivityDataFrame.duplicated(keep=False, subset=phaseActivityDataFrame.columns.difference(variableColumns))]


Activity  Baseline  \
Phase    IDinPhase                                                          
Plan     14                                          Test Audit  REQUIRED   
Develop  13                Mission Based Cyber Risk Assessments  REQUIRED   
         18                                          Test Audit  REQUIRED   
Build    0                                   API Security Tests  REQUIRED   
         14                                          Test Audit  REQUIRED   
Test     0                                   API Security Tests  REQUIRED   
         6                 Mission Based Cyber Risk Assessments  REQUIRED   
         9                   SBOM Software Composition Analysis  REQUIRED   
         11         Software Factory Risk Continuous Monitoring  REQUIRED   
Release  2                 Mission Based Cyber Risk Assessments  REQUIRED   
         5                   SBOM Software Composition Analysis  REQUIRED   
         6          Software Factory Risk Continuous Monitoring  REQUIRED   
Deliver  3                 Mission Based Cyber Risk Assessments  REQUIRED   
         6                   SBOM Software Composition Analysis  REQUIRED   
         7          Software Factory Risk Continuous Monitoring  REQUIRED   
         8                                           Test Audit  REQUIRED   
Deploy   18                  SBOM Software Composition Analysis  REQUIRED   
         19         Software Factory Risk Continuous Monitoring  REQUIRED   
         20                                          Test Audit  REQUIRED   
         21                          User Evaluation / Feedback  REQUIRED   
Operate  5                 Mission Based Cyber Risk Assessments  REQUIRED   
         9                   SBOM Software Composition Analysis  REQUIRED   
         10         Software Factory Risk Continuous Monitoring  REQUIRED   
Monitor  7                 Mission Based Cyber Risk Assessments  REQUIRED   
         14                          User Evaluation / Feedback  REQUIRED   
Feedback 1                           User Evaluation / Feedback  REQUIRED   

                                                                 SSDF  \
Phase    IDinPhase                                                      
Plan     14                          PO.2.1, \nPS 2.1, PW.1.2, PW.2.1   
Develop  13         PW.7.2, RV.1.1, RV.1.2, RV.2.1, RV.3.1, RV.3.2...   
         18                          PO.2.1, \nPS 2.1, PW.1.2, PW.2.1   
Build    0              PO.3.2, \nPO.4.1, \nPW.1.3, \nPW.4.4, \nRV1.2   
         14                      PO.2.1, \nPS 2.1, \nPW.1.2, \nPW.2.1   
Test     0              PO.3.2, \nPO.4.1, \nPW.1.3, \nPW.4.4, \nRV1.2   
         6          PW.7.2, \nRV.1.1, \nRV.1.2, \nRV.2.1, \nRV.3.1...   
         9                                                     PS.3.2   
         11                                                    PS.3.2   
Release  2          PW.7.2, RV.1.1, RV.1.2, RV.2.1, RV.3.1, RV.3.2...   
         5                                                     PS.3.2   
         6                                                     PS.3.2   
Deliver  3          PW.7.2, RV.1.1, RV.1.2, RV.2.1, RV.3.1, RV.3.2...   
         6                                                     PS.3.2   
         7                                                     PS.3.2   
         8                           PO.2.1, \nPS 2.1, PW.1.2, PW.2.1   
Deploy   18                                                    PS.3.2   
         19                                                    PS.3.2   
         20                      PO.2.1, \nPS 2.1, \nPW.1.2, \nPW.2.1   
         21                                                    PO.4.2   
Operate  5          PW.7.2, RV.1.1, RV.1.2, RV.2.1, RV.3.1, RV.3.2...   
         9                                                     PS.3.2   
         10                                                    PS.3.2   
Monitor  7          PW.7.2, \nRV.1.1, \nRV.1.2, \nRV.2.1, \nRV.3.1...   


In [14]:
"   sdfsdf df   ".strip()

'sdfsdf df'

In [15]:
import rdfpandas

In [16]:
graph = rdfpandas.graph.to_graph(phaseActivityDataFrame)

In [17]:
print(graph.serialize(format='turtle'))


"('Build', 0)" "Activity" "API Security Tests" ;
    "Baseline" "REQUIRED" ;
    "Description" "Closed-box test, evaluating the Application Programming Interfaces' compliance with security requirements" ;
    "Inputs" """Executable system/application
Test Plan
Test Cases
Test data""" ;
    "Outputs" "Test results" ;
    "SSDF" """PO.3.2, 
PO.4.1, 
PW.1.3, 
PW.4.4, 
RV1.2""" ;
    "SecurityTestingCM" "Security" ;
    "ToolDependency" "Test tool suite" .

"('Build', 1)" "Activity" "Build" ;
    "Baseline" "REQUIRED" ;
    "Description" "Compile and link" ;
    "Inputs" """Source code;
dependencies""" ;
    "Outputs" """Binary artifacts
Build Report""" ;
    "SSDF" """PO.3.1, 
PO.3.2, 
PO.3.3, 
PO.4.1""" ;
    "SecurityTestingCM" "" ;
    "ToolDependency" """Build tool;
Lint tool;
Artifact repository""" .

"('Build', 2)" "Activity" "Build configuration control and audit" ;
    "Baseline" "REQUIRED" ;
    "Description" """Track build results, SAST and dependency checking report;
""" ;
   

## Experiments

In [ ]:
# files written during google colab session are saved to session storage

f = open("textfile.txt", "a")
f.write("file content!")
f.close()